## Daily feature pipeline

In [ ]:
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

#project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
project_root = "/root/general"
if project_root not in sys.path:
    sys.path.append(project_root)
# Set the environment variables from the file <root_dir>/.env
import config
settings = config.HopsworksSettings(_env_file=f"{project_root}/.env")

HopsworksSettings initialized!


In [ ]:
import datetime
import requests
import pandas as pd
import hopsworks
import util
import datetime
from pathlib import Path
import json
import re
import great_expectations as ge
import warnings
import numpy as np
warnings.filterwarnings("ignore")

### Initialize project

In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 


2025-11-18 12:06:50,735 INFO: Initializing external client
2025-11-18 12:06:50,736 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 12:06:52,218 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279162


Initialize parameters, parameters for papermill otherwise from .env.

In [ ]:
today = datetime.date.today()

# taken from ~/.env. You can also replace settings.AQICN_API_KEY with the api key value as a string "...."
if settings.AQICN_API_KEY is None:
    print("You need to set AQICN_API_KEY either in this cell or in ~/.env")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value() 
country = locals().get("country", "sweden")
city = locals().get("city", "stockholm")
# latitude, longitude = util.get_city_coordinates("stockholm")
latitude, longitude = 59.33, 18.07
print(latitude, longitude)

print(f"Found AQICN_API_KEY: {AQICN_API_KEY}")

secrets = hopsworks.get_secrets_api()
# Replace any existing secret with the new value
secret = secrets.get_secret("AQICN_API_KEY")
if secret is not None:
    secret.delete()
    print("Replacing existing AQICN_API_KEY")

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

File successfully found at the path: /Users/jonaslorenz/Desktop/Code_KTH/ScalableML/AirQualityPrediction/backend/data/rahu-air-quality.csv
59.33 18.07
Found AQICN_API_KEY: 85a05e409148fc36fa8a5aef727bc004a02fa69c
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

Get back the feature groups

In [5]:
air_quality_fg = fs.get_feature_group(
    name='air_quality_with_all_lags',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

Get today's air quality data.

In [ ]:
import yaml
with open(f"{project_root}/sensors.yml", "r") as f:
        config = yaml.safe_load(f)

today_list = []

for sensor in config["sensors"]:
    
    today_list.append(
        util.get_pm25(
            sensor['aqicn_url'], 
            sensor['country'], 
            sensor['city'], 
            sensor['street'], 
            today, 
            AQICN_API_KEY
        )
    )

aq_today_df = pd.DataFrame(today_list)
aq_today_df

,pm25,country,city,street,date,url
0,22.0,sweden,stockholm,stockholm-st-eriksgatan-83,2025-11-18,https://api.waqi.info/feed/@10523
1,11.0,sweden,stockholm,stockholm-hornsgatan-108-gata,2025-11-18,https://api.waqi.info/feed/@10009
2,5.0,sweden,stockholm,hammarby,2025-11-18,https://api.waqi.info/feed/A63637
3,1.0,sweden,stockholm,ljusslingan,2025-11-18,https://api.waqi.info/feed/A65272
4,0.0,sweden,stockholm,sweden-hägersten-älvsjö-stadsdelsområde-nyboho...,2025-11-18,https://api.waqi.info/feed/A474841
5,11.0,sweden,stockholm,sollentuna-sollentunavagen-192,2025-11-18,https://api.waqi.info/feed/@13984
6,0.0,sweden,stockholm,liljeholmen,2025-11-18,https://api.waqi.info/feed/A229291
7,23.0,sweden,stockholm,sundbyberg-tulegatan-9,2025-11-18,https://api.waqi.info/feed/@13988
8,3.0,sweden,stockholm,jultomtestegen,2025-11-18,https://api.waqi.info/feed/A78529
9,2.0,sweden,stockholm,nacka,2025-11-18,https://api.waqi.info/feed/A196735


Get weather forecast data.

In [12]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,-1.70,0.0,7.594208,301.429504,stockholm
1,2025-11-19,-1.30,0.0,4.735060,351.253906,stockholm
2,2025-11-20,1.15,0.1,16.394829,8.841739,stockholm
3,2025-11-21,-3.35,0.1,15.575981,303.690094,stockholm
4,2025-11-22,1.60,0.0,15.463246,245.224884,stockholm
5,2025-11-23,3.45,0.1,10.144082,117.474350,stockholm
6,2025-11-24,2.80,0.0,11.269782,63.435013,stockholm


Compute lag for the last three days and upload today's air quality data into the air quality feature group.

In [32]:
aq_all = air_quality_fg.read()
aq_all["date"] = pd.to_datetime(aq_all["date"]).dt.tz_localize(None)

aq_all = aq_all.sort_values("date").reset_index(drop=True)

lag_feature_list = []

yesterday = datetime.date.today() - datetime.timedelta(days=1)

for sensor in config["sensors"]:

    if today.strftime('%Y-%m-%d') in np.array(
        aq_all[
            (aq_all["country"] == sensor['country']) &
            (aq_all["city"] == sensor['city']) &
            (aq_all["street"] == sensor['street'])
        ]['date']
        .dt
        .strftime('%Y-%m-%d')
        ):
        continue
    
    lag_feature_list.append(
        util.compute_lag_features(
            aq_all,
            sensor['country'], 
            sensor['city'], 
            sensor['street']
        )
    )

lag_feature_frame = pd.DataFrame(lag_feature_list)
print(len(lag_feature_list))
if not len(lag_feature_list) == 0:
    df_to_insert = aq_today_df.merge(lag_feature_frame, on=['country', 'city', 'street'])
    df_to_insert

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.55s) 
11


In [ ]:
if not len(lag_feature_list) == 0:
    df_to_insert['pm25'] = df_to_insert['pm25'].astype('float32')
    air_quality_fg.insert(df_to_insert)

2025-11-17 15:59:26,670 INFO: 	6 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279162/fs/1265774/fg/1717611


Uploading Dataframe: 100.00% |██████████| Rows 11/11 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_with_all_lags_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279162/jobs/named/air_quality_with_all_lags_1_offline_fg_materialization/executions


(Job('air_quality_with_all_lags_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_rolling_2d",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 757786
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 11,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T02:59:26.000670Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type":

Insert today's weather.

In [12]:
weather_fg.insert(daily_df, wait=True)

2025-11-17 15:59:40,292 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279162/fs/1265774/fg/1638074


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279162/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-17 15:59:57,044 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-17 16:00:00,458 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-17 16:02:06,135 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-17 16:02:06,285 INFO: Waiting for log aggregation to finish.
2025-11-17 16:02:18,678 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735353
         }
       },
       "result": {
         "observed_value": 8.121970176696777,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T02:59:40.000292Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": 